In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate
import scipy.optimize

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints

In [ ]:
image_name = '00000B94.png'
image_path = pathlib.Path('.').resolve().parent.joinpath('data', 'images', 'lossless', image_name)

if not image_path.exists():
    url = f'https://zenodo.org/record/3520266/files/{image_name}?download=1'
    urllib.request.urlretrieve(url, image_path)

In [ ]:
img = imageio.imread(image_path)
img = img[:, 1:-1]

In [ ]:
assert img.dtype == np.dtype('uint16')
assert np.shape(img) == (1024, 1022)

In [ ]:
shape = np.shape(img)
x_axis = np.arange(-shape[1]/2, shape[1]/2)/4
y_axis = np.arange(-shape[0]/2, shape[0]/2)/4

In [ ]:
np.shape(x_axis)

In [ ]:
plt.imshow(img)
plt.xlim([400, 600])
plt.ylim([600, 400])
plt.colorbar()

In [ ]:
scale_and_flip = 1 - img[::-1,:] / 2**16
plt.contourf(x_axis, y_axis, scale_and_flip, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
interpolation = scipy.interpolate.RectBivariateSpline(x_axis, y_axis, scale_and_flip.T, kx=1, ky=1)

In [ ]:
x_axis[:, None], y_axis[None, :]

In [ ]:
interpolated = np.swapaxes(interpolation(x_axis, y_axis), 0, 1)
np.shape(interpolated)

In [ ]:
np.allclose(interpolated, scale_and_flip)

In [ ]:
plt.contourf(x_axis, y_axis, interpolated, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
xx, yy = np.meshgrid(x_axis, y_axis)

In [ ]:
np.shape(y_axis[:, None])

In [ ]:
%timeit np.swapaxes(interpolation(x_axis, y_axis), 0, 1)

In [ ]:
%timeit interpolation.ev(np.ravel(xx), np.ravel(yy))

In [ ]:
def create_interpolated_field(x, y, img):
    interpolation = scipy.interpolate.RectBivariateSpline(x, y, img.T, kx=1, ky=1)
    
    def field(x, y):
        if np.shape(x) != np.shape(y):
            raise ValueError("x and y required to be the same shape")
            
        result = interpolation.ev(np.ravel(x), np.ravel(y))
        result.shape = x.shape
        
        return result
    
    return field

In [ ]:
# field = create_interpolated_field(x, y, img)

In [ ]:
interpolated = interpolation.ev(np.ravel(xx), np.ravel(yy))
interpolated.shape = xx.shape

In [ ]:
plt.contourf(x_axis, y_axis, interpolated, 30)
plt.axis('equal')
plt.xlim([-25, 25])
plt.ylim([-25, 25])
plt.colorbar()

In [ ]:
field = pymedphys._mocks.profiles.create_rectangular_field_function([0,0], [10,10], 1, 0)

In [ ]:
np.shape(field(x_axis[:, None], y_axis[None, :]))

In [ ]:
x_centre=-16.45827370949386
y_centre=0.0
x_edge=16.73970349084719
y_edge=15.002190875079133
penumbra=1.1355210572766254
actual_rotation=-88.07256317492568



actual_rotation = actual_rotation
edge_lengths = [x_edge, y_edge]
# penumbra = 0.5
actual_centre = [x_centre, y_centre]

field = pymedphys._mocks.profiles.create_rectangular_field_function(actual_centre, edge_lengths, penumbra, actual_rotation)

In [ ]:
(180 - 90) % 90

In [ ]:
x = np.arange(-50, 50, 0.1)
y = np.arange(-50, 50, 0.1)
xx, yy = np.meshgrid(x, y)
zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
initial_centre = pymedphys._wlutz.findfield._initial_centre(x, y, zz)
centre, rotation = pymedphys._wlutz.findfield.field_finding_loop(field, edge_lengths, penumbra, initial_centre)
centre, rotation

In [ ]:
def draw_by_diff(dx, dy, transform):
    draw_x = np.cumsum(dx)
    draw_y = np.cumsum(dy)

    draw_x, draw_y = pymedphys._wlutz.interppoints.apply_transform(draw_x, draw_y, transform)
    
    return draw_x, draw_y

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.rotate_deg(-rotation)
transform.translate(*centre)

In [ ]:
rotation_x_points = np.linspace(-edge_lengths[0]/2, edge_lengths[0]/2, 51)
rotation_y_points = np.linspace(-edge_lengths[1]/2, edge_lengths[1]/2, 61)

rot_xx_points, rot_yy_points = np.meshgrid(rotation_x_points, rotation_y_points)

rot_xx_points, rot_yy_points = pymedphys._wlutz.interppoints.apply_transform(rot_xx_points, rot_yy_points, transform)

In [ ]:
rect_dx = [-edge_lengths[0]/2, 0, edge_lengths[0], 0, -edge_lengths[0]]
rect_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1], 0]

rect_crosshair_dx = [-edge_lengths[0]/2, edge_lengths[0], -edge_lengths[0], edge_lengths[0]]
rect_crosshair_dy = [-edge_lengths[1]/2, edge_lengths[1], 0, -edge_lengths[1]]

plt.figure(figsize=(10,10))
plt.pcolormesh(xx, yy, zz)
plt.plot(*draw_by_diff(rect_dx, rect_dy, transform), 'k', lw=2)
plt.plot(*draw_by_diff(rect_crosshair_dx, rect_crosshair_dy, transform), 'k', lw=0.5)

# plt.plot(rot_xx_points, rot_yy_points, '.')

plt.scatter(centre[0], centre[1], c='r', s=1)
plt.axis('equal')

In [ ]:
np.allclose(actual_centre, centre)

In [ ]:
np.allclose(actual_rotation, rotation)